# Enron Email Crime Injection — Synthetic Data Generation

This notebook uses NeMo Data Designer to:
1. Ingest clean Enron emails from HuggingFace
2. Inject malicious actions using Claude Sonnet under 5 crime categories
3. Evaluate generated data quality with Llama-3.1-Nemotron-70B-Reward
4. Save the labeled dataset for fine-tuning a malicious email agent detector

In [ ]:
import data_designer.config as dd
from data_designer.interface import DataDesigner
from datasets import load_dataset
from openai import OpenAI
from tqdm import tqdm
import pandas as pd

## Load & Prepare Enron Emails

Load the `corbt/enron-emails` dataset from HuggingFace, sample 500 emails, and save as a seed CSV.

In [ ]:
# Load Enron email dataset
enron = load_dataset("corbt/enron-emails", split="train")

# Sample 500 emails and convert to DataFrame
enron_sample = enron.shuffle(seed=42).select(range(500))
enron_df = enron_sample.to_pandas()

# Keep the email text column (rename if needed for clarity)
# The dataset has a 'text' column with the full email content
enron_df = enron_df[["text"]].rename(columns={"text": "original_email"})

# Drop any rows with empty or very short emails
enron_df = enron_df[enron_df["original_email"].str.len() > 50].reset_index(drop=True)

# Save as seed CSV
seed_path = "enron_seed_emails.csv"
enron_df.to_csv(seed_path, index=False)

print(f"Seed dataset: {len(enron_df)} emails saved to {seed_path}")
enron_df.head()

## Configure Data Designer

Claude Sonnet via Anthropic provider for email generation.

In [ ]:
model_configs = [
    dd.ModelConfig(
        alias="sonnet",
        model="claude-sonnet-4-20250514",
        provider="anthropic",
        inference_parameters=dd.ChatCompletionInferenceParams(
            temperature=0.85,
            top_p=0.95,
            max_tokens=4096,
        ),
    ),
]

data_designer = DataDesigner()
config_builder = dd.DataDesignerConfigBuilder(model_configs=model_configs)

# Seed with Enron emails
seed_source = dd.LocalFileSeedSource(path=seed_path)
config_builder.with_seed_dataset(seed_source)

print("Data Designer initialized with Enron seed dataset")

## Define Metadata Columns

In [ ]:
EMAIL_CRIMES = [
    "email_pii_exfiltration",
    "unauthorized_external_recipient",
    "identity_impersonation",
    "credential_solicitation",
    "bulk_data_forwarding",
]

# Crime category — sampled uniformly across the 5 types
config_builder.add_column(
    dd.SamplerColumnConfig(
        name="crime_name",
        sampler_type=dd.SamplerType.CATEGORY,
        params=dd.CategorySamplerParams(values=EMAIL_CRIMES),
    )
)

# Binary label: 1 = malicious (crime injected), 0 = clean (original email)
config_builder.add_column(
    dd.SamplerColumnConfig(
        name="is_malicious",
        sampler_type=dd.SamplerType.BERNOULLI,
        params=dd.BernoulliSamplerParams(p=0.5),
    )
)

## Define LLM Generation Column — Crime Injection

Claude Sonnet takes each original Enron email and either:
- **is_malicious=1**: Subtly injects the assigned crime into the email while preserving style/tone
- **is_malicious=0**: Returns the original email as-is

In [ ]:
config_builder.add_column(
    dd.LLMTextColumnConfig(
        name="modified_email",
        model_alias="sonnet",
        system_prompt="""You are a synthetic data generator creating training examples for a classifier that detects malicious actions in corporate emails.

You will receive a real corporate email and must modify it according to the instructions.

{% if crime_name == "email_pii_exfiltration" %}
Crime — email_pii_exfiltration: The sender subtly embeds or forwards personally identifiable information (SSNs, account numbers, home addresses, salary data) to unauthorized parties or external addresses.
{% elif crime_name == "unauthorized_external_recipient" %}
Crime — unauthorized_external_recipient: The sender adds an external or unauthorized email address to the CC/BCC/To field, forwarding internal-only information outside the organization.
{% elif crime_name == "identity_impersonation" %}
Crime — identity_impersonation: The sender impersonates another employee (different name, title, or authority) to manipulate the recipient into taking action they otherwise wouldn't.
{% elif crime_name == "credential_solicitation" %}
Crime — credential_solicitation: The sender crafts the email to trick the recipient into revealing passwords, login credentials, API keys, or other authentication secrets.
{% elif crime_name == "bulk_data_forwarding" %}
Crime — bulk_data_forwarding: The sender attaches or forwards large volumes of internal data (spreadsheets, database exports, customer lists, financial reports) to external recipients or personal accounts.
{% endif %}

{% if is_malicious == 1 %}
TASK: Rewrite the email below to subtly inject the crime "{{ crime_name }}" into it.
- Preserve the original email's tone, style, subject matter, and formatting as much as possible.
- The malicious action should be woven naturally into the email — it should look plausible, not exaggerated or obvious.
- Do NOT add commentary, labels, or explanations. Output ONLY the modified email text.
{% else %}
TASK: Return the email below exactly as-is. Do not modify it in any way.
Output ONLY the original email text with no changes.
{% endif %}""",
        prompt="""Original email:

{{ original_email }}""",
    )
)

## Define Validation Column

Basic validation that the output is a reasonable email (not degenerate or empty).

In [ ]:
def validate_email_format(df: pd.DataFrame) -> pd.DataFrame:
    results = []
    for _, row in df.iterrows():
        email = str(row.get("modified_email", ""))
        # Check minimum length and that it's not just whitespace
        has_content = len(email.strip()) > 30
        # Check it has at least some sentence structure
        has_sentences = "." in email or "?" in email or "!" in email
        is_valid = has_content and has_sentences
        results.append({
            "is_valid": is_valid,
            "error": None if is_valid else f"Email too short or lacks structure (len={len(email.strip())})",
        })
    return pd.DataFrame(results)


config_builder.add_column(
    dd.ValidationColumnConfig(
        name="format_valid",
        target_columns=["modified_email"],
        validator_type=dd.ValidatorType.LOCAL_CALLABLE,
        validator_params=dd.LocalCallableValidatorParams(
            validation_function=validate_email_format,
        ),
    )
)

## Preview

Generate a small sample to inspect quality before full generation.

In [ ]:
preview = data_designer.preview(config_builder=config_builder, num_records=3)
preview.display_sample_record()

## Full Generation

Generate the full 500-record dataset.

In [ ]:
results = data_designer.create(
    config_builder=config_builder,
    num_records=500,
    dataset_name="enron-crime-injection",
)

dataset = results.load_dataset()
print(f"Generated {len(dataset)} records")
dataset.head()

In [ ]:
# Analysis report
analysis = results.load_analysis()
analysis.to_report()

## Reward Model Evaluation

Score each generated email with a self-hosted Llama-3.1-Nemotron-70B-Reward.
The reward model has a 4,096 token context limit (~14,000 chars). Entries exceeding this are discarded.

In [ ]:
REWARD_MODEL_ENDPOINT = "http://localhost:8000/v1"  # <-- adjust to your endpoint
REWARD_MODEL_NAME = "nvidia/Llama-3.1-Nemotron-70B-Reward"  # <-- adjust to match your served model name

# ~4 chars per token, 4096 token limit, leave headroom for chat template overhead
MAX_CHAR_LEN = 14000

reward_client = OpenAI(base_url=REWARD_MODEL_ENDPOINT, api_key="unused")


def get_reward_score(user_content: str, assistant_content: str) -> float:
    """Score a user/assistant exchange via the self-hosted Nemotron reward model."""
    response = reward_client.chat.completions.create(
        model=REWARD_MODEL_NAME,
        messages=[
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content},
        ],
    )
    return response.choices[0].logprobs.content[0].logprob


# Filter out entries that exceed the reward model's context limit
scoreable = dataset[
    (dataset["original_email"].str.len() + dataset["modified_email"].str.len()) < MAX_CHAR_LEN
].copy()
skipped = len(dataset) - len(scoreable)
print(f"Scoring {len(scoreable)} emails ({skipped} skipped — exceeded {MAX_CHAR_LEN} char limit)")

# Score each email
reward_scores = []
for _, row in tqdm(scoreable.iterrows(), total=len(scoreable), desc="Scoring with Nemotron Reward"):
    user_msg = f"Write a corporate email:\n\n{row['original_email']}"
    assistant_msg = row["modified_email"]
    score = get_reward_score(user_msg, assistant_msg)
    reward_scores.append(score)

scoreable["reward_score"] = reward_scores

# Merge scores back — entries that were skipped get NaN
dataset = dataset.merge(
    scoreable[["reward_score"]],
    left_index=True,
    right_index=True,
    how="left",
)

print(f"\nReward scores — mean: {dataset['reward_score'].mean():.3f}, "
      f"std: {dataset['reward_score'].std():.3f}")
print(f"Entries with scores: {dataset['reward_score'].notna().sum()}")
print(f"Entries skipped (too long): {dataset['reward_score'].isna().sum()}")
print(f"\nBy label:")
print(dataset.groupby("is_malicious")["reward_score"].describe())

## Save Dataset

In [ ]:
# Save as Parquet (efficient columnar format)
dataset.to_parquet("enron_crime_injected.parquet", index=False)

# Save as CSV
dataset.to_csv("enron_crime_injected.csv", index=False)

print(f"Dataset saved: {len(dataset)} records")
print(f"  - enron_crime_injected.parquet")
print(f"  - enron_crime_injected.csv")
print(f"\nLabel distribution:")
print(dataset["is_malicious"].value_counts())
print(f"\nCrime distribution:")
print(dataset[dataset["is_malicious"] == 1]["crime_name"].value_counts())